In [8]:
# Imports

import cv2
from PIL import Image

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import math as maths

import os
import random

from sklearn.model_selection import train_test_split

import torch
import torch.nn as nn
from torch.nn import functional
#from torchsummary import summary
#import torchvision.transforms as transforms

from tqdm import tqdm
from IPython.display import clear_output

import wandb
import pprint

import io
import sys
sys.path.append('../../')

from functions import import_imagedata, get_data, label_oh_tf,  Unwrap, ImageProcessor
from architectures import vgg16net, smallnet1, smallnet2, smallnet3
from loop_fns import loop, test_loop
from fns4wandb import build_optimizer, set_optimizer, hp_sweep, train_model, train_log, log_test_score, set_lossfn, pipeline

from architectures import build_net, smallnet3

wandb.login()

True

In [9]:
""""
grid search
'learning_rate': {
            'values': [6.1e-5, 6.2e-5, 6.3e-5, 6.4e-5]
"""

'"\ngrid search\n\'learning_rate\': {\n            \'values\': [6.1e-5, 6.2e-5, 6.3e-5, 6.4e-5]\n'

In [15]:
config = {
    'method': 'random',
    'metric':{
        'goal': 'minimize',
        'name': 'val_loss'},
    'parameters': {
        'dropout':{
            'values': [0.5, 0.4, 0.3]
        },
        'weight_decay':{
            'values': [1e-5,2e-5, 3e-5,4e-5]
        },
        'epochs':{
            'value': 120
        },
        'lin_layer_size': {
            'values': [100] #, 150, 50
        },
        'first_lin_lay':{
            'values':[267264] #1x267264 and 1055232x100)
        },
        'optimizer': {
            'values': ['adam']
        },
            'learning_rate': {
                # a flat distribution between 0 and 0.1
                'distribution': 'log_uniform_values',
                'min': 5e-5,
                'max': 1e-4
            },
        'scheduler': {
            'values': [0.2]
        },
        'loss_fn': {
            'values': ['MSE', 'CrossEntropy']
        },
        'data_set':{
            'values':['Augmented']
        },
        'num_classes' : {
            'values':[11]
        },
        'ks': {
            'values': [(3,5)]
        },
        'model_name' : {'values': ['smallnet3']},
        'channels' : {'values': [3]},
        'image_path': {
            'values': [r'/its/home/nn268/antvis/optics/AugmentedDS_IDSW/']
        }
    }
}

col_dict = {
    'colour': 'colour',
    'size': [226,72],
    'padding': 5
}

title = f'IDSW_unwrap_2c2l_112023'

save_dict = {'Run' : title,
            'Current_Epoch': 0}


sweep_id = wandb.sweep(config, project=f"SanityCheck_modelSize_2conv2lin_IDSW_aug_{col_dict['colour']}_{col_dict['size']}")

Create sweep with ID: 2kfrzfie
Sweep URL: https://wandb.ai/antvis/SanityCheck_modelSize_2conv2lin_IDSW_aug_colour_%5B226%2C%2072%5D/sweeps/2kfrzfie


In [ ]:

"""config = dict(
    model_name = {'values': ['smallnet3']},
    epochs= 80, 
    learning_rate =8e-5,
    dataset= 'IDSW_Aug',
    architecture ='CNN',
    optimizer= 'adam',
    loss_fn = 'CrossEntropy',
    weight_decay= 4e-5,
    dropout = 0.4,
    first_lin_lay =1064448,
    lin_layer_size= 100,
    ks = [3,5],
    in_chan = 3,
    num_classes =11,
    scheduler = 0.2
)"""
"""config = {
     "model_name" : 'smallnet3',
    "epochs" :80, 
    "learning_rate" : 8e-5,
    "dataset" : 'IDSW_Aug',
    "architecture" :'CNN',
    "optimizer": 'adam',
    "loss_fn" : 'CrossEntropy',
    "weight_decay": 4e-5,
    "dropout" : 0.4,
    "first_lin_lay" :1055232,
    "lin_layer_size": 100,
    "ks" : [3,5],
    "in_chan" : 3,
    "num_classes" :11,
    "scheduler" : 0.2,
    "channels": 3
     
 }



col_dict = {
    'colour': 'colour',
    'size': [452,144],
    'padding': 5,
    'model_size': '2c2l'
}
"""

In [ ]:
def tr(config=None):
    with wandb.init(config=config):
        config = wandb.config
        model = hp_sweep(config, col_dict, save_dict)
        
wandb.agent(sweep_id, tr, count=25)

wandb: Agent Starting Run: 1v6jxnme with config:
wandb: 	channels: 3
wandb: 	data_set: Augmented
wandb: 	dropout: 0.3
wandb: 	epochs: 120
wandb: 	first_lin_lay: 267264
wandb: 	image_path: /its/home/nn268/antvis/optics/AugmentedDS_IDSW/
wandb: 	ks: [3, 5]
wandb: 	learning_rate: 5.613717279046053e-05
wandb: 	lin_layer_size: 100
wandb: 	loss_fn: CrossEntropy
wandb: 	model_name: smallnet3
wandb: 	num_classes: 11
wandb: 	optimizer: adam
wandb: 	scheduler: 0.2
wandb: 	weight_decay: 3e-05


/its/home/nn268/.local/lib/python3.8/site-packages/torch/nn/modules/container.py:217: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


improvment in metrics. model saved


In [ ]:
#wandb.agent(sweep_id, tr, count=25)

In [ ]:
model = #pipeline(config, col_dict, title="2c2l_training_113x36, 3chan", image_file_path= "/its/home/nn268/antvis/optics/AugmentedDS_IDSW/")